In [1]:
import cv2
import numpy as np



In [2]:
ratio_th = 0.85

In [3]:
# Load images
img_color_1 = cv2.imread('spy1.jpg')
img_color_2 = cv2.imread('spy2.jpg')


img1 = cv2.cvtColor(img_color_1, cv2.COLOR_BGR2YCrCb)[:,:,0]
img2 = cv2.cvtColor(img_color_2, cv2.COLOR_BGR2YCrCb)[:,:,0]

# Create SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
keypoints2, descriptors2 = sift.detectAndCompute(img2, None)

# Create a matcher
bf = cv2.BFMatcher()

# Match descriptors
matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# Apply ratio test to filter good matches
good_matches = []
for m, n in matches:
    if m.distance < ratio_th * n.distance:
        good_matches.append(m)

# Draw matches on images
img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)


# Display the result
win = cv2.namedWindow('Initially found matches', cv2.WINDOW_NORMAL)
cv2.imshow('Initially found matches', img_matches)


In [5]:
descriptors1

array([[ 11.,   6.,   8., ...,   5.,   1.,   0.],
       [ 10.,   0.,   0., ...,   1.,   1.,   3.],
       [ 25.,   1.,   1., ...,   5.,   4.,   2.],
       ...,
       [103.,   0.,   0., ...,   1.,   0.,   0.],
       [138.,   6.,   0., ...,   5.,   0.,   0.],
       [114.,  32.,   2., ...,   5.,   4.,   0.]], dtype=float32)

In [4]:
# Extract matched keypoints
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Use RANSAC to estimate the transformation matrix
transformation_matrix, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

print(transformation_matrix)
# Draw matches on images excluding outliers
img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, 
                              np.array(good_matches)[mask.ravel() == 1], 
                              None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
win = cv2.namedWindow('Matches after reprojection iterations', cv2.WINDOW_NORMAL)
cv2.imshow('Matches after reprojection iterations', img_matches)


# Apply the transformation to image1
registered_image = cv2.warpPerspective(img_color_1, transformation_matrix, (img_color_2.shape[1], img_color_2.shape[0]))

# Display the results
cv2.imshow('Registered Image 1', registered_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 8.28459077e-01  1.74744141e-01  1.24715582e+02]
 [-5.58416519e-01  2.69695916e+00 -2.19695505e+02]
 [-1.23571585e-03  2.09700626e-03  1.00000000e+00]]
